## Python Descriptor

- Data descriptor: define `__get__()` and `__set__()`
- Non-data descriptor: define `__set__()`
- Read Only data descriptor: define both `__get__()` and `__set__()`, with `__set__()` raising an AttributeError

- Descriptor can be called directly by its method name `d.__get__(obj)`
- `obj.d` looks up d in `obj.__dict__`, if `d` defines `__get__()`, then `d.__get__(obj)` invoked before `obj.__dict__[d]`
- For objects
    - `object.__getattribute__()` transforms `b.x` into `type(b).__dict__('x').__get__(b, type(b)), full implementation in `PyObject_GenericGetAttr()` in `Objects/object.c`
- For classes
    - `type.__getattribute__()` transforms `B.x` into `B.__dict__['x'].__get__(None, B)`

In [5]:
def __getattribute__(self, key):
    "Emulate type_getattro() in Objects/typeobject.c"
    v = object.__getattribute__(self, key)
    if hasattr(v, '__get__'):
        return v.__get__(None, self)
    return v

In [7]:
class RevealAccess(object):
    """A data descriptor that sets and return values
        normally and prints a message logging their access.
    """
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name
    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val
    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val
class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
m = MyClass()
print(m.x)
m.x = 20
m.y

Retrieving var "x"
10
Updating var "x"


5

In [1]:
class Property(object):
    """Emulate PyProperty_Type() in Objects/descrobject.c"""
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("Unreadable attribute.")
        return self.fget(obj)
    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("Can't set attribute.")
        self.fset(obj, value)
    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

- Simulating a spreadsheet

In [3]:
class Cell(object):
    ...
    def getvalue(self):
        "Recalculate the cell before returning value."
        self.recalc()
        return self._value
    value = property(getvalue)

- To support method calls, functions include `__get__()` method for binding methods during attribute access. 
- All functions are descriptors return bound and unbound methods depending whether invoked from a object or a class.

In [8]:
class Functions(object):
    ...
    def __get__(self, obj, objtype=None):
        'Simulate func_descr_get() in Objects/funcobject.c'
        return types.MethodType(self, obj, objtype)

In [10]:
class D(object):
    def f(self, x):
        return x
d = D()
print(D.__dict__['f'])    # Stored internally as a function
print(D.f)    # Get from a class becomes an unbound method(In Python3 No unbound method)
print(d.f)    # Get from a instance becomes a bound method

<function D.f at 0x10b51ae18>
<function D.f at 0x10b51ae18>
<bound method D.f of <__main__.D object at 0x10b5791d0>>


- The actual C implementation of `PyMethod_Type` in `Objects/classobject.c` is a single object with two diffrent representations depending on whether the `im_self` is set or is NULL

Transformation|Called from an Object|Called from a Class
--|--|--
function|f(obj, *args)|f(*args)
staticmethod|f(*args)|f(*args)
classmethod|f(type(obj), *args)|f(klass, *args)

In [11]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc

- One use for classmethods is to create alternate class constructors.

In [ ]:
class Dict(object):
    ...
    def fromkeys(klass, iterable, value=None):
        "Emulate dict_fromkeys() in Objects/dictobject.c"
        d = klass()
        for key in iterable:
            d[key] = value
        return d
    fromkeys = classmethod(fromkeys)